## TODO
* now using app ImplicitSession, because aiovk can't auth from user side. Probably need to auth throw vk-api and then use aiovk.TokenSession

### Experiment to asynchonously get users friends using aiovk lib

In [102]:
from typing import List, Optional

import aiovk
import asyncio
import aiohttp
import vk_api
import requests
import json

In [157]:
login = input()
password = input()
proxy_address = input()
proxy_port = input()

 79687655667
 LKUAjTW9
 45.95.96.187
 8746


In [131]:
s = requests.Session()
user_agent = 'Mozilla/5.0 (Windows NT 6.1; rv:52.0) Gecko/20100101 Firefox/52.0'
s.headers.update({'User-agent': user_agent})
for proxy_protocol in ["http", "https"]:
    s.proxies.update({proxy_protocol: f"{proxy_address}:{proxy_port}"})
    
vk_api_session = vk_api.VkApi(login, password, session=s)
vk_api_session.auth()

SyntaxError: EOL while scanning string literal (<ipython-input-131-baf8aa9d4011>, line 5)

In [40]:
access_token = vk_api_session.token["access_token"]

In [158]:
proxy = aiovk.drivers.ProxyDriver(proxy_address, proxy_port)
session = aiovk.TokenSession(access_token, driver=proxy)

In [44]:
async_vk_api = aiovk.API(session)

In [79]:
await async_vk_api.users.get(user_ids=1)

[{'first_name': 'Павел', 'id': 1, 'last_name': 'Дуров'}]

In [88]:
async def get_friends(user_id, async_api):
    return await async_api.friends.get(user_id=user_id)

In [112]:
await get_friends(491221043, async_vk_api)

{'count': 12,
 'items': [11522454,
  31035858,
  35059537,
  138581643,
  142478661,
  164415202,
  199879596,
  222803967,
  237527395,
  266523052,
  268845403,
  365071613]}

In [97]:
with open("../resources/checkpoints/data_checkpoint.json") as f:
    user_data = json.load(f)
groups_to_get = user_data["68076353"]["groups"]
del user_data

async def get_group_wall(group_id, async_api):
    return await async_api.wall.get(owner_id=-1 * group_id, extended=False) # negative value means group, positive - user

/usr/lib/python3.8/json/decoder.py:353: RuntimeWarning: coroutine 'Request.__call__' was never awaited
  obj, end = self.scan_once(s, idx)


### Asynchronous way

In [161]:
class TokenSessionWithProxy:
    """wrapper for AsyncVkExecuteRequestPool from aiovk because it doesn't support passing
    Driver to session"""
    def __init__(self, proxy_address, proxy_port):
        self.proxy_address, self.proxy_port = proxy_address, proxy_port
    
    def __call__(self, token):
        proxy_driver = aiovk.drivers.ProxyDriver(self.proxy_address, self.proxy_port)
        return aiovk.TokenSession(token, driver=proxy_driver)


In [164]:
from aiovk.pools import AsyncVkExecuteRequestPool

async def get_groups_walls(groups_ids: List[int], access_token, pool_size=25, req_params: Optional[dict]=None):
    # !!!: TODO: should not await for one pool execute, need to make all executes() simultaneously and wait for results
    if req_params is None:
        req_params = {}
        
    groups_ids = groups_ids.copy()
    responses = []
    pool_executes = []
    
    while groups_ids:
        pool = AsyncVkExecuteRequestPool(token_session_class=TokenSessionWithProxy(proxy_address, proxy_port))
        
        for _ in range(pool_size):
            if not groups_ids:
                break
            group_id = groups_ids.pop(0)
            resp = pool.add_call("wall.get", access_token, {"owner_id": -1 * group_id, **req_params})
            responses.append(resp)
            
        pool_executes.append(pool.execute())
        
    await asyncio.gather(*pool_executes)
    return responses

In [169]:
from time import time

start_time = time()
groups_walls = await get_groups_walls(groups_to_get, access_token=session.access_token, req_params={"extended": False})
print("time needed", time() - start_time)

RuntimeError: asyncio.run() cannot be called from a running event loop

In [173]:
groups_walls[0].result

{'count': 99326,
 'items': [{'id': 10763636,
   'from_id': -58170807,
   'owner_id': -58170807,
   'date': 1623252720,
   'marked_as_ads': 0,
   'post_type': 'post',
   'text': 'Heпocpeдcтвeннo Kaxa. Жeнитьбa (2021) 16+ 🔥 \nЖaнp: кoмeдия \n \nB гocти к Kaxe пpиeзжaeт eгo двoюpoднaя cecтpa Шyшaник — бaбyшкa peшилa ocтaвить eй в нacлeдcтвo cвoю квapтиpy. Чтoбы нe пoтepять нacлeдcтвo, Kaxa peшaeт жeнить Шyшaник нa cвoeм дpyгe Cepгo.',
   'is_pinned': 1,
   'attachments': [{'type': 'photo',
     'photo': {'album_id': -7,
      'date': 1623250101,
      'id': 457458953,
      'owner_id': -58170807,
      'has_tags': False,
      'access_key': 'c0230a230682a4e8ba',
      'height': 1065,
      'photo_1280': 'https://sun6-20.userapi.com/impg/n2cDFhmJpCiJuUUkaWGXv_37d8M27v9lo_Enaw/2T6od0B-nQs.jpg?size=1280x717&quality=96&sign=b998b11e7c630f65f634713f9a8d4437&c_uniq_tag=NKLzNvwD3O9AqOCrYZTVX1oW1HU-b2Hx7CSiY2zS1AQ&type=album',
      'photo_130': 'https://sun6-20.userapi.com/impg/n2cDFhmJpCiJuUUka

### synchronous way

In [119]:
pool = vk_api.requests_pool.VkRequestsPool(vk_api_session)

In [124]:
%%time

responses = []
for idx, group in enumerate(groups_to_get):
    resp_raw = pool.method("wall.get", values={"owner_id": -1 * group, "extended": False})
    responses.append(resp_raw)
    if idx % 25 == 24:
        pool.execute()

CPU times: user 595 ms, sys: 78.9 ms, total: 674 ms
Wall time: 25.5 s
